# Pipelines utilizados para obtener la información

## 1. PIPELINE 1: Contenido de videos de YouTube

Este pipeline se utiliza para extraer el contenido (Lo hablado) de los videos de YouTube que contienen información sobre los organismos autonomos y la reforma para la Ley del Poder Judicial. Es importante mencionar que este pipeline no recupera los titulos de los videos, la descripción de los videos o los comentarios de los videos. Unicamente tiene la función de extraer el contenido de videos de YouTube.

### Proceso de ejecución del pipeline:

1. Utilizando la herramiento de octoparse se puede efectuar web scraping para obtener la lista de videos de YouTube que contienen la información sobre los organismos autónomos y la reforma para la Ley del Poder Judicial. Un ejemplo de uso es que al incluir la palabra clave "organismos autónomos". Octoparse despues de un proceso de **casi 7 minutos**, logro recopilar 219 videos de youtube que hablan del tema. 

### Resumen del pipeline:
1. Obtener listado de videos con **octoparse**.
2. Quedarme unicamente con las urls y guardarlas en un csv de una unica columna.
3. Descargar audio de videos de youtube utlizando **yp_dlp** y la **lista de urls**
4. Obtener el corpus de cada uno de los audios utilizando **speech_recognition**
5. Unir corpus en un solo archivo

THE BELOW TOOL CAN BE USED TO DOWNLOAD AUDIO FROM YOUTUBE VIDEOS.
- This tool use yt_dlp library to download audio from YouTube videos.

In [ ]:
import yt_dlp
from pathlib import Path
import pandas as pd
import time
from datetime import datetime
import csv
import re
import unicodedata

def normalize_title(title: str) -> str:
    """
    Normalizes a title following specified rules:
    1. Converts to lowercase
    2. Replaces spaces with underscores
    3. Removes accents
    4. Removes non-allowed characters (only allows letters, numbers and underscores)
    
    Args:
        title (str): Original title to normalize
        
    Returns:
        str: Normalized title
    """
    # Convert to lowercase and Replace spaces with underscores
    title = title.lower()
    title = title.replace(' ', '_')
    
    # Remove accents
    title = ''.join(
        c for c in unicodedata.normalize('NFKD', title)
        if not unicodedata.combining(c)
    )
    
    # Remove non-allowed characters (only keeps letters, numbers and underscores)
    title = re.sub(r'[^a-z0-9_]', '', title)
    
    # Remove multiple consecutive underscores and Remove underscores at the beginning and end
    title = re.sub(r'_+', '_', title)
    
    title = title.strip('_')
    
    return title

class BatchAudioDownloader:
    def __init__(self):
        self.success_count = 0
        self.failed_count = 0
        self.failed_urls = []

    def download_audio(self, url: str, output_path: str) -> dict:
        """
        Descarga el audio de un video de YouTube
        
        Args:
            url (str): URL del video de YouTube
            output_path (str): Ruta donde se guardará el audio
            
        Returns:
            dict: Resultado de la descarga con status y mensaje
        """
        try:
            # Crear el directorio de salida si no existe
            output_dir = Path(output_path)
            output_dir.mkdir(parents=True, exist_ok=True)
            
            # Configuración para la descarga
            ydl_opts = {
                'format': 'bestaudio/best',
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'mp3',
                    'preferredquality': '192',
                }],
                'outtmpl': str(output_dir / '%(title)s.%(ext)s'),
                'quiet': False,
                'no_warnings': True
            }
            
            # Realizar la descarga
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                # Obtener información del video primero
                info = ydl.extract_info(url, download=False)
                video_title = info.get('title', 'Unknown Title')
                
                # Normalizar el título
                normalized_title = normalize_title(video_title)
                
                # Actualizar la configuración con el nuevo título normalizado
                ydl_opts['outtmpl'] = str(output_dir / f'{normalized_title}.%(ext)s')
                
                print(f"\nDescargando: {video_title}")
                print(f"Nombre del archivo: {normalized_title}.mp3")
                
                # Crear nueva instancia con la configuración actualizada
                with yt_dlp.YoutubeDL(ydl_opts) as ydl_download:
                    ydl_download.download([url])
                
                return {
                    'status': 'success',
                    'message': f'Audio descargado exitosamente: {normalized_title}',
                    'title': normalized_title,
                    'url': url,
                    'output_path': str(output_dir)
                }
                
        except Exception as e:
            return {
                'status': 'error',
                'message': f'Error durante la descarga: {str(e)}',
                'url': url
            }

    def process_csv(self, csv_path: str, output_path: str):
        """
        Procesa un archivo CSV con URLs de YouTube y descarga los audios
        
        Args:
            csv_path (str): Ruta al archivo CSV
            output_path (str): Ruta donde se guardarán los audios
        """
        try:
            # Leer el CSV sin encabezados
            urls = pd.read_csv(csv_path, header=None)[0].tolist()
            total_urls = len(urls)
            
            print(f"\nIniciando proceso de descarga de {total_urls} videos...")
            
            # Crear archivo de log
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            log_file = Path(output_path) / f'download_log_{timestamp}.csv'
            
            with open(log_file, 'w', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(['URL', 'Status', 'Title', 'Error'])
                
                # Procesar cada URL
                for index, url in enumerate(urls, 1):
                    print(f"\nProcesando {index}/{total_urls}: {url}")
                    
                    # Intentar descargar
                    result = self.download_audio(url, output_path)
                    
                    # Actualizar contadores y log
                    if result['status'] == 'success':
                        self.success_count += 1
                        writer.writerow([url, 'Success', result['title'], ''])
                    else:
                        self.failed_count += 1
                        self.failed_urls.append(url)
                        writer.writerow([url, 'Failed', '', result['message']])
                    
                    # Pequeña pausa entre descargas
                    time.sleep(1)
            
            return {
                'total': total_urls,
                'success': self.success_count,
                'failed': self.failed_count,
                'log_file': str(log_file)
            }
            
        except Exception as e:
            print(f"Error al procesar el CSV: {str(e)}")
            return None

def main():
    # Rutas de entrada y salida
    csv_path = "C:/git/IAClass/15_projectU4_reforma_judical_org_autonomos/data_scraped/youtube_URLs/urls_organismos_autonomos_v01.csv"
    output_path = "C:/git/IAClass/15_projectU4_reforma_judical_org_autonomos/data_scraped/youtube_audio/"
    
    # Crear instancia del descargador y procesar el CSV
    downloader = BatchAudioDownloader()
    results = downloader.process_csv(csv_path, output_path)
    
    # Mostrar resumen
    if results:
        print("\n=== Resumen de Descargas ===")
        print(f"Total de URLs procesadas: {results['total']}")
        print(f"Descargas exitosas: {results['success']}")
        print(f"Descargas fallidas: {results['failed']}")q
        print(f"Archivo de log: {results['log_file']}")
        
        if downloader.failed_urls:
            print("\nURLs que fallaron:")
            for url in downloader.failed_urls:
                print(f"- {url}")

if __name__ == "__main__":
    main()

THE BELOW TOOL CAN BE USED TO GET AUDIO TEXT FROM AUDIO FILES.
THIS TOOL USE:
- SpeechRecognition library for Python to recognize speech.
- Pydub library for Python to manipulate audio files.

FOR GET TEXT FORM LARGE AUDIO FILES, THIS TOOL CAN BE USED TO SEGMENT THE FILES TO SMALLER SEGMENTS (Specificly 60sgs by segment)

In [ ]:
import os
import speech_recognition as sr
from pydub import AudioSegment
import threading
from queue import Queue
from typing import List

class AudioTranscriber:
    def __init__(self, input_folder: str, output_folder: str, max_threads: int = 8, language: str = 'es-ES'):
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.max_threads = max_threads
        self.language = language
        self.audio_queue = Queue()
        self.active_threads = []
        self.thread_semaphore = threading.Semaphore(max_threads)
        
        # Crear carpeta de salida si no existe
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

    def prepare_voice_file(self, path: str) -> str:
        if os.path.splitext(path)[1] == '.wav':
            return path
        elif os.path.splitext(path)[1] in ('.mp3', '.m4a', '.ogg', '.flac'):
            audio_file = AudioSegment.from_file(
                path, format=os.path.splitext(path)[1][1:])
            wav_file = os.path.splitext(path)[0] + '.wav'
            audio_file.export(wav_file, format='wav')
            return wav_file
        else:
            raise ValueError(
                f'Unsupported audio format: {format(os.path.splitext(path)[1])}')

    def segment_audio(self, audio_path: str, segment_length: int = 45000):
        audio = AudioSegment.from_file(audio_path)
        segments = []
        for i in range(0, len(audio), segment_length):
            segment = audio[i:i+segment_length]
            segments.append(segment)
        return segments

    def transcribe_audio(self, audio_data, language) -> str:
        print(f'[Thread-{threading.current_thread().name}] Transcribiendo segmento de audio...')
        r = sr.Recognizer()
        try:
            text = r.recognize_google(audio_data, language=language)
            return text
        except sr.RequestError as e:
            print(f"No se pudieron obtener resultados del servicio de reconocimiento de voz de Google; {e}")
        except sr.UnknownValueError:
            print("Google Speech Recognition no pudo entender el audio")
        except Exception as e:
            print(f"Ocurrió un error: {e}")
        return ""

    def write_transcription_to_file(self, text: str, input_file: str) -> None:
        # Obtener el nombre base del archivo de audio
        base_name = os.path.splitext(os.path.basename(input_file))[0]
        output_file = os.path.join(self.output_folder, f"{base_name}.txt")
        
        print(f'[Thread-{threading.current_thread().name}] Escribiendo transcripción en {output_file}')
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(text)

    def process_single_file(self, input_file: str) -> None:
        try:
            with self.thread_semaphore:
                print(f'[Thread-{threading.current_thread().name}] Procesando: {input_file}')
                
                # Preparar el archivo
                wav_file = self.prepare_voice_file(input_file)
                segments = self.segment_audio(wav_file)
                full_transcription = ""

                # Procesar cada segmento
                for i, segment in enumerate(segments):
                    print(f'[Thread-{threading.current_thread().name}] Procesando segmento {i+1} de {len(segments)}...')
                    segment_file = f"temp_segment_{threading.current_thread().name}_{i}.wav"
                    segment.export(segment_file, format="wav")

                    with sr.AudioFile(segment_file) as source:
                        audio_data = sr.Recognizer().record(source)
                        text = self.transcribe_audio(audio_data, self.language)
                        full_transcription += text + " "

                    # Limpieza del archivo temporal
                    if os.path.exists(segment_file):
                        os.remove(segment_file)

                # Guardar la transcripción
                self.write_transcription_to_file(full_transcription.strip(), input_file)

                # Limpieza del archivo WAV si fue convertido
                if wav_file != input_file and os.path.exists(wav_file):
                    os.remove(wav_file)

        except Exception as e:
            print(f'[Thread-{threading.current_thread().name}] Error procesando {input_file}: {str(e)}')

    def get_audio_files(self) -> List[str]:
        """Obtiene la lista de archivos de audio soportados en la carpeta de entrada."""
        supported_formats = ('.wav', '.mp3', '.m4a', '.ogg', '.flac')
        audio_files = []
        
        for file in os.listdir(self.input_folder):
            if file.lower().endswith(supported_formats):
                audio_files.append(os.path.join(self.input_folder, file))
        
        return audio_files

    def process_all_files(self):
        """Procesa todos los archivos de audio en la carpeta de entrada usando hilos."""
        audio_files = self.get_audio_files()
        
        if not audio_files:
            print("No se encontraron archivos de audio soportados en la carpeta especificada.")
            return

        print(f"Se encontraron {len(audio_files)} archivos para procesar.")
        
        # Crear y empezar hilos para cada archivo
        for audio_file in audio_files:
            thread = threading.Thread(target=self.process_single_file, args=(audio_file,))
            thread.start()
            self.active_threads.append(thread)

        # Esperar a que todos los hilos terminen
        for thread in self.active_threads:
            thread.join()

        print("Procesamiento completado para todos los archivos.")

if __name__ == '__main__':
    # Ejemplo de uso
    input_folder = "C:/git/IAClass/15_projectU4_reforma_judical_org_autonomos/data_scraped/youtube_audio/"
    output_folder = "C:/git/IAClass/15_projectU4_reforma_judical_org_autonomos/data_scraped/youtube_corpus_from_audio/"
    
    transcriber = AudioTranscriber(
        input_folder=input_folder,
        output_folder=output_folder,
        max_threads=8,
        language='es-ES'
    )
    
    transcriber.process_all_files()